# Vertex Similarity
----

In this notebook, we will use cuGraph to compute vertex similarity using both the Jaccard Similarity and the Overlap Coefficient.  


Notebook Credits

    Original Authors: Bradley Rees
    Last Edit: 08/13/2019

RAPIDS Versions: 0.9.0

Test Hardware
* GV100 32G, CUDA 10.0



## Introduction




### Defining a Set
Both Jaccard and the Overlap Coefficient operate on sets, and in a graph setting, those sets are the list of neighbor vertices. <br>
For those that like math:  The neighbors of a vertex, _v_, is defined as the set, _U_, of vertices connected by way of an edge to vertex v, or _N(v) = {U} where v ∈ V and ∀ u ∈ U ∃ edge(v,u)∈ E_.

For the rest of this introduction, set A will equate to A = N(i) and set B will quate to B = N(j).  That just make the rest of the text more readable.

### Jaccard Similarity

The Jaccard similarity between two sets is defined as the ratio of the volume of their intersection divided by the volume of their union. 

The Jaccard Similarity can then be defined as

<a href="https://www.codecogs.com/eqnedit.php?latex=js(A,B)&space;=&space;\frac{|A&space;\cap&space;B|}{|A&space;\cup&space;B&space;|&space;}&space;=&space;\frac{|A&space;\cap&space;B|}{&space;|A|&space;&plus;&space;|B|&space;-&space;|A&space;\cup&space;B&space;|&space;}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?js(A,B)&space;=&space;\frac{|A&space;\cap&space;B|}{|A&space;\cup&space;B&space;|&space;}&space;=&space;\frac{|A&space;\cap&space;B|}{&space;|A|&space;&plus;&space;|B|&space;-&space;|A&space;\cup&space;B&space;|&space;}" title="js(A,B) = \frac{|A \cap B|}{|A \cup B | } = \frac{|A \cap B|}{ |A| + |B| - |A \cup B | }" /></a>



For further detail see Wikipedia - https://en.wikipedia.org/wiki/Jaccard_index

To compute the Jaccard similarity between all pairs of vertices connected by an edge in cuGraph use: <br>
__jaccard(G)__

    G: A cugraph.Graph object

Returns:

    df: cudf.DataFrame with three names columns:
        df["source"]: The source vertex id.
        df["destination"]: The destination vertex id.
        df["jaccard_coeff"]: The jaccard coefficient computed between the source and destination vertex.

<br>


__References__

    https://research.nvidia.com/publication/2017-11_Parallel-Jaccard-and

### Overlap Coefficient

The Overlap Coefficient between two sets is defined as the ratio of the volume of their intersection divided by the volume of the smaller set.
The Overlap Coefficient can be defined as

<a href="https://www.codecogs.com/eqnedit.php?latex=oc(A,B)&space;=&space;\frac{|A|&space;\cap&space;|B|}{min(|A|,&space;|B|)&space;}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?oc(A,B)&space;=&space;\frac{|A&space;\cap&space;B|}{min(|A|,&space;|B|)&space;}" title="oc(A,B) = \frac{|A \cap B|}{min(|A|, |B|) }" /></a>

For further detail see Wikipedia - https://en.wikipedia.org/wiki/Overlap_coefficient

To compute the Overlap Coefficient between all pairs of vertices connected by an edge in cuGraph use: <br>

__overlap(G)__

    G: A cugraph.Graph object

Returns:

    df: cudf.DataFrame with three names columns:
        df["source"]: The source vertex id.
        df["destination"]: The destination vertex id.
        df["overlap_coeff"]: The overlap coefficient computed between the source and destination vertex.



## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

In [ ]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

----
### Define some Print functions
(the `del` are not needed since going out of scope should free memory)

In [ ]:
# define a function for printing the top most similar vertices
def print_most_similar_jaccard(df):
    
    jmax = df['jaccard_coeff'].max()
    dm = df.query('jaccard_coeff >= @jmax')    
    
    #find the best
    for i in range(len(dm)):    
        print("Vertices " + str(dm['source'][i]) + " and " + 
              str(dm['destination'][i]) + " are most similar with score: " 
              + str(dm['jaccard_coeff'][i]))
    del jmax
    del dm

In [ ]:
# define a function for printing the top most similar vertices
def print_most_similar_overlap(df):
    
    smax = df['overlap_coeff'].max()
    dm = df.query('overlap_coeff >= @smax')      
    
    for i in range(len(dm)):
        print("Vertices " + str(dm['source'][i]) + " and " + 
          str(dm['destination'][i]) + " are most similar with score: " 
          + str(dm['overlap_coeff'][i]))
        
    del smax
    del dm

In [ ]:
# define a function for printing jaccard similar vertices based on a threshold
def print_jaccard_threshold(_d, limit):
    
    filtered = _d.query('jaccard_coeff > @limit')
    
    for i in range(len(filtered)):
        print("Vertices " + str(filtered['source'][i]) + " and " + 
            str(filtered['destination'][i]) + " are similar with score: " + 
            str(filtered['jaccard_coeff'][i]))

In [ ]:
# define a function for printing similar vertices based on a threshold
def print_overlap_threshold(_d, limit):
    
    filtered = _d.query('overlap_coeff > @limit')
    
    for i in range(len(filtered)):
        if filtered['source'][i] != filtered['destination'][i] :
            print("Vertices " + str(filtered['source'][i]) + " and " + 
                str(filtered['destination'][i]) + " are similar with score: " + 
                str(filtered['overlap_coeff'][i]))

### Read the CSV datafile using cuDF
data file is actually _tab_ separated, so we need to set the delimiter

In [ ]:
# Test file  
datafile='./data/karate-data.csv'

# define the column names
cols = ["src", "dst"]

# define the column data types
dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [ ]:
# Let's look at the DataFrame. There should be two columns and 156 records
gdf

In [ ]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

### Create a Graph

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [ ]:
G.degree()

In [ ]:
# How many vertices are in the graph?  Remember that Graph is zero based
G.number_of_vertices()

_The test graph has only 34 vertices, so why is the Graph listing 35?_

As mentioned above, cuGraph vertex numbering is zero-based, meaning that the first vertex ID starts at zero.  The test dataset is 1-based.  Because of that, the Graph object adds an extra isolated vertex with an ID of zero.  Hence the difference in vertex count.  
We are working on a renumbering feature to address this issue.  

### Jaccard 

In [ ]:
# Call cugraph.nvJaccard 
%time df = cugraph.jaccard(G)

The Most similar shoul be 33 and 34.
Vertex 33 has 12 neighbors, vertex 34 has 17 neighbors.  They share 10 neighbors in common:
$jaccard = 10 / (10 + (12 -10) + (17-10)) = 10 / 19 = 0.526$

In [ ]:
print_most_similar_jaccard(df)

In [ ]:
### let's look at all similarities over a threshold
print_jaccard_threshold(df, 0.4)

In [ ]:
# Since it is a small graph we can print all scores.
# Notice that only connected vertices are computed

# let's sort the data first.  Please note that you may get a warning.  Just ignore it.  
## It is just converted into a dataframe so that we could do this function call.  
## If we were going to actually do further work on it, we would leave it as it was :)

g = df.groupby(['jaccard_coeff'], method='cudf', as_index=False)
df_s = g.as_df()


In [ ]:
# The groupby as_df function returns a tuple where the first item is the dataframe
print_jaccard_threshold(df_s[0], 0.0)

### Overlap Coefficient

In [ ]:
# Call cugraph.nvJaccard 
do = cugraph.overlap(G)

In [ ]:
print_most_similar_overlap(do)

### Expanding vertex pairs for similarity scoring

In [ ]:
# get all two-hop vertex pairs
p = G.get_two_hop_neighbors()

In [ ]:
# Let's look at the Jaccard score
j2 = cugraph.jaccard(G, first=p['first'], second=p['second'])

In [ ]:
print_most_similar_jaccard(j2)

notice that there are a lot of very similar vertices.  For example vertices 15 and 16 share their only two neighbors in common.  

In [ ]:
j2o = cugraph.overlap(G, first=p['first'], second=p['second'])

In [ ]:
print_most_similar_overlap(j2o)

the overlap score captures all the same matches that Jaccrd did, but also includes those sets that are exact subsets

----
### Adjusting the vertex ID
Let's adjust all the vertex IDs to be zero based.  We are going to do this by adding two new columns with the adjusted IDs

In [ ]:
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [ ]:
# create a new Graph 
G2 = cugraph.Graph()
G2.add_edge_list(gdf["src_0"], gdf["dst_0"])

In [ ]:
# How many vertices are in the graph?  Remember that Graph is zero based while teh data start at vertex 1
G2.number_of_vertices()

The number of vertices now matches what is in the test graph

In [ ]:
# Call cugraph.nvJaccard 
df2 = cugraph.jaccard(G2)

In [ ]:
print_most_similar_jaccard(df2)

Adjusting the vertices back (e.g adding +1 to vertex IDs) yields 33 and 34 which matches the orginal results.
For Jaccard, the fact that vertex IDs do not start of 0 is not an issue

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___